# Checkpointing Models

In [1]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [3]:
# path to output model
model_path = "./output/model_persistence/"

In [4]:
# load the training and testing data, then scale it into the range [0, 1]
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...


In [5]:
# convert the labels from integers to vectors
labelBinarizer = LabelBinarizer()
trainY = labelBinarizer.fit_transform(trainY)
testY = labelBinarizer.fit_transform(testY)

# Improve Checkpointing

In [6]:
from classifiers.mini_vgg import MiniVGGNet

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
miniVGG = MiniVGGNet()
model = miniVGG.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
fname = os.path.sep.join([model_path, "minivgg-weights-{epoch:03d}-{val_loss:.4f}.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
callbacks = [checkpoint]

In [10]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), 
              batch_size=64, epochs=40, 
              callbacks=callbacks, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 11s 215us/step - loss: 0.5632 - acc: 0.8016 - val_loss: 0.6082 - val_acc: 0.7892

Epoch 00001: val_loss improved from inf to 0.60823, saving model to ./output/model_persistence//minivgg-weights-001-0.6082.hdf5
Epoch 2/40
50000/50000 [==============================] - 11s 217us/step - loss: 0.5420 - acc: 0.8087 - val_loss: 0.5775 - val_acc: 0.8019

Epoch 00002: val_loss improved from 0.60823 to 0.57755, saving model to ./output/model_persistence//minivgg-weights-002-0.5775.hdf5
Epoch 3/40
50000/50000 [==============================] - 11s 217us/step - loss: 0.5183 - acc: 0.8162 - val_loss: 0.5882 - val_acc: 0.7960

Epoch 00003: val_loss did not improve from 0.57755
Epoch 4/40
50000/50000 [==============================] - 11s 218us/step - loss: 0.4959 - acc: 0.8246 - val_loss: 0.5866 - val_acc: 0.8047

Epoch 00004: val_loss did not improve from 0.57755
E

50000/50000 [==============================] - 11s 224us/step - loss: 0.2181 - acc: 0.9229 - val_loss: 0.5765 - val_acc: 0.8263

Epoch 00040: val_loss did not improve from 0.55345


# Best Checkpointing

In [11]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
fname = os.path.sep.join([model_path, "minivgg-weights-best.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
callbacks = [checkpoint]

In [12]:
H = model.fit(trainX, trainY, validation_data=(testX, testY), 
              batch_size=64, epochs=40, 
              callbacks=callbacks, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 11s 214us/step - loss: 0.2143 - acc: 0.9236 - val_loss: 0.5947 - val_acc: 0.8233

Epoch 00001: val_loss improved from inf to 0.59468, saving model to ./output/model_persistence//minivgg-weights-best.hdf5
Epoch 2/40
50000/50000 [==============================] - 11s 227us/step - loss: 0.2135 - acc: 0.9235 - val_loss: 0.5719 - val_acc: 0.8276

Epoch 00002: val_loss improved from 0.59468 to 0.57188, saving model to ./output/model_persistence//minivgg-weights-best.hdf5
Epoch 3/40
50000/50000 [==============================] - 12s 231us/step - loss: 0.2090 - acc: 0.9247 - val_loss: 0.5766 - val_acc: 0.8280

Epoch 00003: val_loss did not improve from 0.57188
Epoch 4/40
50000/50000 [==============================] - 11s 224us/step - loss: 0.2092 - acc: 0.9244 - val_loss: 0.5873 - val_acc: 0.8245

Epoch 00004: val_loss did not improve from 0.57188
Epoch 5/40
50000/50000 [================

Well Done